In [26]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
import numpy as np
from scipy import sparse
import nltk
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.multioutput import MultiOutputClassifier
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
from collections import Counter
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Required Methods

In [480]:
"""Read the input XML CLinical files"""
# Function to read and extract data from XML files
def extract_data_from_xml(folder_path):
    # Initialize a list to store patient records and annotations
    data = []

    # List files in the folder
    files = os.listdir(folder_path)

    # Iterate over each XML file
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        #print(f"Processing XML file: {file_name}")

        # Parse the XML file
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Extract text content from the TEXT element
        text_content = root.find('.//TEXT').text.strip()

        # Create a dictionary to store the patient record
        record = {"text": text_content}

        # Loop through each TAG element
        for tag in root.findall('.//TAGS/*'):
            attribute = tag.tag
            value = tag.attrib.get('met', None)  # Check if 'met' attribute exists
            record[attribute] = value

        # Append the record to the list of data
        data.append(record)

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(data)

    return df

In [2]:
""" Class having all the necessary features for classfication """

class TextData:
    def __init__(self, text_id, preprocessed_text, word_embedding, target_labels, ner_tags,weighted_embedding):
        self.text_id = text_id
        self.preprocessed_text = preprocessed_text
        self.word_embedding = word_embedding
        self.target_labels = target_labels  # List of labels or one-hot encoded vector
        self.ner_tags = ner_tags  
        self.weighted_embedding=weighted_embedding
        self.entity_presence=None

In [4]:
"""Text Preprocessing"""

def clean_text(text):
    pattern = r"[\n\t]\d+"  # Capture newline or tab followed by one or more digits
    return re.sub(pattern, "", text)

def remove_special_characters(text):
    # Define a regex pattern to match special symbols, newline, and tab characters
    pattern = r'[^a-zA-Z0-9\s]'  # This pattern matches any character that is not alphanumeric or whitespace

    # Use re.sub() to replace all occurrences of the pattern with an empty string
    cleaned_text = re.sub(pattern, '', text)

    # Additionally, remove newline and tab characters
    cleaned_text = cleaned_text.replace('\n', '').replace('\t', '')

    return cleaned_text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    # Join tokens back into a single string
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

# Define a function to remove numbers if their length is greater than 3
def remove_long_numbers(text):
    # Define the pattern to match numbers of length greater than 3
    pattern = r'\b\d{4,}\b'
    # Remove the matched numbers
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text


### Named Entities Recognition using Clinical-AI-Apollo/Medical-NER Transformer

In [5]:
from transformers import pipeline

def predict_entities_df_pipeline(df, text_column="preprocessed_text"):

    # Create pipeline for clinical NER
    pipe = pipeline("token-classification", model="Clinical-AI-Apollo/Medical-NER", aggregation_strategy='simple')

    # Apply pipeline to each text in the dataframe
    df['predicted_entities'] = df[text_column].apply(lambda x: pipe(x))
    return df

C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [6]:
def Label_encode_target(column_labels,datadf):
    # Initialize the label encoder
    encoder = LabelEncoder()
    
    
    # Met:0 , Not met :1
    for column in data_df.columns:
        # Exclude the column named "Text"
        if column != 'text':
            # Encode the values of the current column
            data_df[column] = encoder.fit_transform(data_df[column])
    
    # Find counts for each column
    for column in data_df.columns:
        if column != 'text':
            class_counts = data_df[column].value_counts()
            #print(f"Counts for {column}:")
            #print(class_counts)
        #print()

In [7]:
def create_entity_features(entities, le):
    """
    This function takes a list of entity dictionaries and returns a feature vector.
    """
    # Initialize feature vector with zeros
    features = np.zeros_like(le.classes_)  
    entity_types = [entity['entity_group'] for entity in entities if entity['entity_group']]  # Filter out empty entity types
    
    for entity_type in entity_types:
        # Get the index of the entity type using label encoding
        entity_type_idx = le.transform([entity_type])[0]
        # Set the corresponding element to 1 to indicate presence of entity
        features[entity_type_idx] = 1
    
    # Set elements to zero for empty entity types
    empty_entity_types = set(le.classes_) - set(entity_types)
    for empty_entity_type in empty_entity_types:
        entity_type_idx = le.transform([empty_entity_type])[0]
        features[entity_type_idx] = 0
    
    return features


In [8]:
def tfidf_weighted_average(document, word_embeddings, tfidf_weights, default_embedding=None):

    word_counts = Counter(document.split())
    total_words = len(word_counts)  # Count of unique words

    if default_embedding is None:
        default_embedding = np.zeros_like(next(iter(word_embeddings.values())))  # Use the first embedding vector as the default

    vocabulary = set(word_counts.keys())

    # Ensure that the vocabulary of word_embeddings matches the vocabulary of the document
    if set(word_embeddings.keys()) != vocabulary:
        raise ValueError("Vocabulary of word_embeddings doesn't match the vocabulary of the document")

    # Calculate weighted sum of word embeddings
    weighted_sum = np.zeros_like(default_embedding)
    for word, count in word_counts.items():
        weighted_sum += tfidf_weights[word] * word_embeddings.get(word, default_embedding)

    # Calculate mean of weighted word embeddings
    average_embedding = weighted_sum / total_words

    return average_embedding


from gensim.models import Word2Vec

def generate_word_embedding(text):
    # Tokenize the text into words
    tokenized_text = text.split()
    # Initialize and train the Word2Vec model
    model = Word2Vec([tokenized_text], min_count=1, vector_size=100)  
    # Generate word embeddings for the current text using the trained model
    word_embeddings = {word: model.wv[word] for word in tokenized_text}
    return word_embeddings


### Starting the execution of files/input

In [481]:

# Mount Google Drive
#drive.mount('/content/drive')

# Path to the folder containing the XML files
folder_path = r'C:\Users\aslam\Downloads\train\train'

# Extract data from XML files and create a DataFrame
data_df = extract_data_from_xml(folder_path)

In [482]:
data_df.head()

,text,ABDOMINAL,ADVANCED-CAD,ALCOHOL-ABUSE,ASP-FOR-MI,CREATININE,DIETSUPP-2MOS,DRUG-ABUSE,ENGLISH,HBA1C,KETO-1YR,MAJOR-DIABETES,MAKES-DECISIONS,MI-6MOS
0,Record date: 2106-02-12\n\nCampbell Orthopedic...,not met,met,not met,met,not met,met,not met,met,not met,not met,met,met,met
1,Record date: 2079-05-12\n\n\n\n\n\nMERCY CARE ...,not met,met,not met,met,not met,not met,not met,met,not met,not met,not met,met,not met
2,Record date: 2120-09-19\n\nPersonal Data and O...,met,met,not met,met,not met,not met,not met,met,not met,not met,met,met,not met
3,Record date: 2067-11-24\n\n ...,not met,met,not met,met,not met,not met,not met,met,not met,not met,not met,met,not met
4,Record date: 2094-02-16\n\nJENNIFER BOOKER\n\n...,not met,met,not met,not met,not met,not met,not met,met,met,not met,met,met,met


In [484]:
data_df['preprocessed_text'] = data_df['text'].apply(clean_text)

data_df['preprocessed_text'] = data_df['preprocessed_text'].apply(preprocess_text)

data_df['preprocessed_text'] = data_df['preprocessed_text'].apply(remove_special_characters)

data_df['preprocessed_text'] = data_df['preprocessed_text'].apply(remove_long_numbers)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\User

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\User

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\User

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\User

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\User

...

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\User

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\User

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\User

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\User

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\User

In [567]:
data_df1=data_df.copy()
data_df.head()

,text,ABDOMINAL,ADVANCED-CAD,ALCOHOL-ABUSE,ASP-FOR-MI,CREATININE,DIETSUPP-2MOS,DRUG-ABUSE,ENGLISH,HBA1C,KETO-1YR,MAJOR-DIABETES,MAKES-DECISIONS,MI-6MOS,preprocessed_text
0,Record date: 2106-02-12\n\nCampbell Orthopedic...,not met,met,not met,met,not met,met,not met,met,not met,not met,met,met,met,record date campbell orthopedic associate mad...
1,Record date: 2079-05-12\n\n\n\n\n\nMERCY CARE ...,not met,met,not met,met,not met,not met,not met,met,not met,not met,not met,met,not met,record date mercy care center mercy internal ...
2,Record date: 2120-09-19\n\nPersonal Data and O...,met,met,not met,met,not met,not met,not met,met,not met,not met,met,met,not met,record date personal data overall health pati...
3,Record date: 2067-11-24\n\n ...,not met,met,not met,met,not met,not met,not met,met,not met,not met,not met,met,not met,record date huntington emergency dept visit t...
4,Record date: 2094-02-16\n\nJENNIFER BOOKER\n\n...,not met,met,not met,not met,not met,not met,not met,met,met,not met,met,met,met,record date jennifer booker lc unit nashua m...


In [568]:
data_df.set_index(data_df.index, inplace=True)
data_df.index.name = 'text_id'

In [14]:
from collections import Counter

# Calculate TF-IDF weights for each document
def calculate_tfidf_weights(documents):
    tfidf_weights = []
    for doc in documents:
        word_counts = Counter(doc.split())
        total_words = len(word_counts)

        # Calculate IDF weights
        idf_weights = {word: np.log(total_words / word_counts.get(word, 1)) for word in word_counts}

        # Calculate TF-IDF weights
        tfidf_weights.append({word: (count / total_words) * idf_weights[word] for word, count in word_counts.items()})

    return tfidf_weights

data_df = data_df.reset_index()
text_data = []

target_columns = ['ABDOMINAL', 'ADVANCED-CAD', 'ALCOHOL-ABUSE', 'MAJOR-DIABETES', 'DRUG-ABUSE']

# Calculate TF-IDF weights for all documents
tfidf_weights = calculate_tfidf_weights(data_df['preprocessed_text'])

# Loop through only the first 5 rows
for index, row in data_df.iterrows():
    text_id = row['text_id']
    preprocessed_text = row['preprocessed_text']
    
    # Word Embeddings - Word2Vec Model
    word_embedding = generate_word_embedding(preprocessed_text)
    weighted_embedding = tfidf_weighted_average(preprocessed_text, word_embedding, tfidf_weights[index])  # Provide the corresponding TF-IDF weights
    
    # Reset target_labels for each row
    target_labels = []
    for target_column in target_columns:
        target_labels.append(row[target_column])
    
    # NER prediction function
    predicted_entities = predict_entities_df_pipeline(pd.DataFrame({'preprocessed_text': [preprocessed_text]}))['predicted_entities'].tolist()[0]
    
    text_data.append(TextData(text_id, preprocessed_text, word_embedding, target_labels, predicted_entities, weighted_embedding))


C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

In [15]:
unique_entity_types = set()
for text_object in text_data:
    unique_entity_types.update(entity['entity_group'] for entity in text_object.ner_tags)

unique_entity_types_list = list(unique_entity_types)
if len(unique_entity_types_list) > 0:
    # Fit the label encoder with all unique entity types encountered
    le = LabelEncoder()
    le.fit(unique_entity_types_list)
else:
    print("No unique entity types found. Skipping label encoder fitting.")

In [16]:
for text_object in text_data:
    entity_presence = create_entity_features(text_object.ner_tags, le)
    text_object.entity_presence = entity_presence

In [17]:
type(text_data[1].entity_presence)

numpy.ndarray

In [19]:
def save_text_data_to_txt(data_list, filename):
    """Saves a list of TextData objects to a text file in a human-readable format.

    Args:
        data_list (list): A list of TextData objects.
        filename (str): The name of the text file to save to.
    """

    with open(filename, 'w') as f:
        for data in data_list:
            # Handle potential issues with word_embedding, target_labels, predicted_entities, and entity_presence
            word_embedding_str = None
            if isinstance(data.word_embedding, list):
                word_embedding_str = ','.join(map(str, data.word_embedding))  # Join list elements
            elif isinstance(data.word_embedding, str):
                word_embedding_str = data.word_embedding  # Already a string
            elif isinstance(data.word_embedding, np.ndarray):
                # Convert NumPy array to comma-separated string
                word_embedding_str = ','.join(map(str, data.word_embedding.tolist()))
            elif isinstance(data.word_embedding, dict):
                # Convert dictionary to comma-separated string
                word_embedding_str = ','.join(map(str, data.word_embedding.values()))
            else:
                raise TypeError(f"Unsupported data type for word_embedding: {type(data.word_embedding)}")

            weighted_embedding_str = None
            if isinstance(data.weighted_embedding, np.ndarray):
                # Convert NumPy array to comma-separated string
                weighted_embedding_str = ','.join(map(str, data.weighted_embedding.tolist()))
            else:
                raise TypeError(f"Unsupported data type for weighted_embedding: {type(data.weighted_embedding)}")

            target_labels_str = None
            if isinstance(data.target_labels, list):
                target_labels_str = ','.join(data.target_labels)  # Join list elements
            elif isinstance(data.target_labels, str):
                target_labels_str = data.target_labels  # Already a string
            else:
                raise TypeError(f"Unsupported data type for target_labels: {type(data.target_labels)}")

            predicted_entities_str = None
            if isinstance(data.ner_tags, list):
                predicted_entities_str = ','.join(map(str, data.ner_tags))  # Join list elements
            elif isinstance(data.ner_tags, str):
                predicted_entities_str = data.ner_tags  # Already a string
            else:
                # Handle case where predicted_entities is a dictionary (same logic as before)
                if isinstance(data.ner_tags, dict):
                    entity_labels = ','.join(data.ner_tags.values())
                    predicted_entities_str = f"predicted_entities: {entity_labels}"  # Include label

            entity_presence_str = None
            if isinstance(data.entity_presence, np.ndarray):
                # Convert NumPy array to comma-separated string
                entity_presence_str = ','.join(map(str, data.entity_presence.tolist()))
            else:
                raise TypeError(f"Unsupported data type for entity_presence: {type(data.entity_presence)}")

            # Write data to the file in a clear format
            f.write(f"text_id: {data.text_id}\n")
            f.write(f"preprocessed_text: {data.preprocessed_text}\n")
            if word_embedding_str:
                f.write(f"word_embedding: {word_embedding_str}\n")
            if weighted_embedding_str:
                f.write(f"weighted_embedding: {weighted_embedding_str}\n")
            if target_labels_str:
                f.write(f"target_labels: {target_labels_str}\n")
            if predicted_entities_str:
                f.write(f"Entities: {predicted_entities_str}\n")
            if entity_presence_str:
                f.write(f"entity_presence: {entity_presence_str}\n\n")

# Example usage
save_text_data_to_txt(text_data, "text_data_final2.txt")


In [24]:
len(text_data[0].word_embedding)

1363

In [25]:
len(text_data[0].weighted_embedding)

100

In [183]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
encoder = LabelEncoder()

# Define label columns
label_cols = ['ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES','ADVANCED-CAD'] 

# Met:0 , Not met :1
for column in label_cols:
        data_df[column] = encoder.fit_transform(data_df[column])


In [250]:
train, test = train_test_split(data_df, random_state=42, test_size=0.20, shuffle=True)

label_cols = ['ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES','ADVANCED-CAD']

# Use columns argument in drop with set difference
y_train = train.drop(columns=list(set(train.columns) - set(label_cols)))

# Use columns argument in drop with set difference
y_test = test.drop(columns=list(set(test.columns) - set(label_cols)))

In [410]:
test.iloc[45]

text_id                                                            241
text                 Record date: 2078-10-25\n\n \n\n \n\n \n\nOcto...
ABDOMINAL                                                            1
ADVANCED-CAD                                                         1
ALCOHOL-ABUSE                                                  not met
ASP-FOR-MI                                                         met
CREATININE                                                           0
DIETSUPP-2MOS                                                      met
DRUG-ABUSE                                                     not met
ENGLISH                                                            met
HBA1C                                                          not met
KETO-1YR                                                       not met
MAJOR-DIABETES                                                       0
MAKES-DECISIONS                                                    met
MI-6MO

In [251]:
def compute_average_embedding(word_embeddings):
    # Compute the average of all word embeddings
    embedding_dim = len(next(iter(word_embeddings.values())))
    avg_embedding = np.zeros(embedding_dim)
    for embedding in word_embeddings.values():
        avg_embedding += embedding
    avg_embedding /= len(word_embeddings)
    return avg_embedding

# Convert text_data into a single array of weighted embeddings
X_avg_embed = np.array([compute_average_embedding(doc.word_embedding)for doc in text_data])

### Calculating Features : For the rest of the implementations the Features are vectorized text and TF IDF weighted embeddings

In [252]:
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()

# Feature Engineering - TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000) 
tfidf_features = tfidf_vectorizer.fit_transform([doc.preprocessed_text for doc in text_data])

# Convert weighted embedding and entity presence to NumPy arrays
X_weighted_embedding = np.array([doc.weighted_embedding for doc in text_data])
#X_entity_presence = np.array([doc.entity_presence for doc in text_data])

X_weighted_embedding = X_weighted_embedding.astype(np.float64)
#X_entity_presence = X_entity_presence.astype(np.float64)
tfidf_features = tfidf_features.astype(np.float64)

#X_weighted_embedding_normalized = scaler.fit_transform(X_weighted_embedding)
# Stack features horizontally
#X_combined = hstack((X_weighted_embedding, X_entity_presence, tfidf_features))

X_combined = hstack((X_weighted_embedding, tfidf_features))

## Classification

###  Random Forest and SVM :  MultiOutputClassifier

In [253]:

y = data_df['ADVANCED-CAD']

# Split data into training and testing sets with stratification
X_train, X_test, _, _ = train_test_split(X_combined,y , test_size=0.2, random_state=42)

# Multi-output classification with MultiOutputClassifier
svm_model = MultiOutputClassifier(SVC(kernel='poly'))  
svm_model.fit(X_train, y_train)

# Predict with the SVM model
y_pred_svm = svm_model.predict(X_test)

# Multi-output classification with MultiOutputClassifier for Random Forest
rf_model = MultiOutputClassifier(RandomForestClassifier(n_estimators=200,max_depth=10))  
rf_model.fit(X_train, y_train)

# Evaluate Random Forest Model
y_pred_rf = rf_model.predict(X_test)
rf_report = classification_report(y_test, y_pred_rf)

print("Random Forest Classification Report:\n", rf_report)

# Evaluate SVM Model
y_pred_svm = svm_model.predict(X_test)
svm_report = classification_report(y_test, y_pred_svm)

print("SVM Classification Report:\n", svm_report)


C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.97      0.76        34
           1       0.82      0.33      0.47        27
           2       0.71      1.00      0.83        35
           3       0.63      0.39      0.48        31

   micro avg       0.67      0.70      0.69       127
   macro avg       0.70      0.67      0.64       127
weighted avg       0.69      0.70      0.65       127
 samples avg       0.66      0.70      0.64       127

SVM Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.88      0.74        34
           1       0.78      0.52      0.62        27
           2       0.69      0.89      0.78        35
           3       0.68      0.48      0.57        31

   micro avg       0.68      0.71      0.69       127
   macro avg       0.70      0.69      0.68       127
weighted avg       0.69      0.71      0.68       127
 samples av

C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [254]:
# Evaluate Random Forest Model
y_pred_rf = rf_model.predict(X_train)
rf_accuracy = accuracy_score(y_train, y_pred_rf)
rf_report = classification_report(y_train, y_pred_rf)

# Print Random Forest Results
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest Classification Report:\n", rf_report)

# Evaluate SVM Model
y_pred_svm = svm_model.predict(X_train)
svm_accuracy = accuracy_score(y_train, y_pred_svm)
svm_report = classification_report(y_train, y_pred_svm)

# Print SVM Results
print("SVM Accuracy:", svm_accuracy)
print("SVM Classification Report:\n", svm_report)

C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Random Forest Accuracy: 1.0
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       142
           1       1.00      1.00      1.00        86
           2       1.00      1.00      1.00       142
           3       1.00      1.00      1.00        97

   micro avg       1.00      1.00      1.00       467
   macro avg       1.00      1.00      1.00       467
weighted avg       1.00      1.00      1.00       467
 samples avg       0.94      0.94      0.94       467

SVM Accuracy: 1.0
SVM Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       142
           1       1.00      1.00      1.00        86
           2       1.00      1.00      1.00       142
           3       1.00      1.00      1.00        97

   micro avg       1.00      1.00      1.00       467
   macro avg       1.00      1.00      1.00       467
weighted avg       

C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [572]:
###  Random Forest  : BinaryRelevance

In [259]:
from skmultilearn.problem_transform import BinaryRelevance
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(RandomForestClassifier(n_jobs=-1))

# train
classifier.fit(X_train, y_train)

# predict
predictions = classifier.predict(X_test)

rf_binary = classification_report(y_test, predictions)
print(rf_binary)

              precision    recall  f1-score   support

           0       0.61      0.97      0.75        34
           1       0.89      0.30      0.44        27
           2       0.69      0.94      0.80        35
           3       0.60      0.29      0.39        31

   micro avg       0.66      0.65      0.66       127
   macro avg       0.70      0.63      0.60       127
weighted avg       0.69      0.65      0.61       127
 samples avg       0.65      0.67      0.61       127



C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVC( Kernel:Poly) - ClassifierChain

In [314]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain

# initialize classifier chains multi-label classifier
classifier = ClassifierChain(SVC(kernel='poly'))

# Training logistic regression model on train data
classifier.fit(X_train, y_train)

# predict
predictions = classifier.predict(X_test)
predictions_t = classifier.predict(X_train)
svm_c = classification_report(y_test, predictions)
svm_c2 = classification_report(y_train, predictions_t)
print(svm_c)
print(svm_c2)

              precision    recall  f1-score   support

           0       0.64      0.88      0.74        34
           1       0.81      0.48      0.60        27
           2       0.65      0.89      0.75        35
           3       0.77      0.74      0.75        31

   micro avg       0.69      0.76      0.72       127
   macro avg       0.72      0.75      0.71       127
weighted avg       0.71      0.76      0.72       127
 samples avg       0.64      0.71      0.64       127

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       142
           1       1.00      0.57      0.73        86
           2       0.80      1.00      0.89       142
           3       0.99      0.71      0.83        97

   micro avg       0.92      0.86      0.89       467
   macro avg       0.95      0.82      0.86       467
weighted avg       0.94      0.86      0.88       467
 samples avg       0.84      0.76      0.79       467



C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVC( Kernel:Linear) - ClassifierChain

In [415]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain

# initialize classifier chains multi-label classifier
classifier = ClassifierChain(SVC(kernel='linear',C=0.7,gamma=0.7))

# Training logistic regression model on train data
classifier.fit(X_train, y_train)

# predict
predictions = classifier.predict(X_test)
predictions_t = classifier.predict(X_train)
svm_c = classification_report(y_test, predictions)
svm_c2 = classification_report(y_train, predictions_t)
print(svm_c)
print(svm_c2) 

              precision    recall  f1-score   support

           0       0.60      1.00      0.75        34
           1       0.83      0.37      0.51        27
           2       0.66      0.94      0.78        35
           3       0.66      0.87      0.75        31

   micro avg       0.65      0.82      0.72       127
   macro avg       0.69      0.80      0.70       127
weighted avg       0.68      0.82      0.71       127
 samples avg       0.62      0.81      0.67       127

              precision    recall  f1-score   support

           0       0.66      1.00      0.80       142
           1       0.98      0.72      0.83        86
           2       0.81      1.00      0.89       142
           3       0.70      0.96      0.81        97

   micro avg       0.75      0.94      0.83       467
   macro avg       0.79      0.92      0.83       467
weighted avg       0.77      0.94      0.83       467
 samples avg       0.73      0.88      0.77       467



C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
predictions = classifier.predict(X_test)

In [392]:
predictions

<57x4 sparse matrix of type '<class 'numpy.float64'>'
	with 160 stored elements in Compressed Sparse Column format>

In [401]:
predictions_l = predictions.toarray()
predictions_l[45]

array([1., 1., 1., 0.])

In [428]:
import numpy as np

# Sample data array
data_array = np.array([[1, 0, 1, 0]])

# Label columns
label_cols = ['ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES', 'ADVANCED-CAD']

# Convert array to list
data_list = data_array.tolist()[0]  # Get first row as a list

# Create a dictionary for mapping 
label_mapping = {col: 0 for col in label_cols}

# Decode the list based on mapping
decoded_list = []
for i, value in enumerate(data_list):
    if value == 0:
        decoded_list.append(label_cols[i])  # Append label if condition met
    else:
        decoded_list.append("Not Met")

# Print the decoded list
print(decoded_list)


['Not Met', 'CREATININE', 'Not Met', 'ADVANCED-CAD']


#### saving the .pkl file

In [412]:
import joblib

classifier = ClassifierChain(SVC(kernel='linear',C=0.7,gamma=0.7))
classifier.fit(X_train, y_train)

# Save the model
joblib.dump(classifier, 'SVC.pkl')  

['SVC.pkl']

In [319]:
import joblib

classifier = ClassifierChain(SVC(kernel='linear',C=0.7,gamma=0.7))
classifier.fit(X_train, y_train)

# Save the model
joblib.dump(classifier, 'SVM.pkl')  

['SVM.pkl']

### RandomForestClassifier - ClassifierChain

In [422]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain

# initialize classifier chains multi-label classifier
classifier = ClassifierChain(RandomForestClassifier())

# Training logistic regression model on train data
classifier.fit(X_train, y_train)

# predict
predictions = classifier.predict(X_test)
predictions_t = classifier.predict(X_train)
svm_c = classification_report(y_test, predictions)
svm_c2 = classification_report(y_train, predictions_t)
print(svm_c)
print(svm_c2)

              precision    recall  f1-score   support

           0       0.62      0.97      0.76        34
           1       0.83      0.37      0.51        27
           2       0.71      1.00      0.83        35
           3       0.47      0.26      0.33        31

   micro avg       0.66      0.68      0.67       127
   macro avg       0.66      0.65      0.61       127
weighted avg       0.66      0.68      0.62       127
 samples avg       0.65      0.69      0.62       127

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       142
           1       1.00      1.00      1.00        86
           2       1.00      1.00      1.00       142
           3       1.00      1.00      1.00        97

   micro avg       1.00      1.00      1.00       467
   macro avg       1.00      1.00      1.00       467
weighted avg       1.00      1.00      1.00       467
 samples avg       0.94      0.94      0.94       467



C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\aslam\Anaconda3\Anaconda\Li

In [427]:
import joblib

classifier = ClassifierChain(RandomForestClassifier())
classifier.fit(X_train, y_train)

# Save the model
joblib.dump(classifier, 'SVC1.pkl')

['SVC1.pkl']

In [179]:
X_train.shape

(225, 1100)

### Multi layer Neural Network model

In [189]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),  
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='sigmoid')  # Use 'sigmoid' activation for multi-label classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use binary crossentropy for multi-label classification
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train,
                    epochs=10,  # Adjust the number of epochs as needed
                    batch_size=32,  # Adjust batch size as needed
                    validation_split=0.2)  # Use a portion of training data for validation

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Epoch 1/10


C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.3438 - loss: 0.6920 - val_accuracy: 0.6667 - val_loss: 0.6832
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5135 - loss: 0.6827 - val_accuracy: 0.6667 - val_loss: 0.6729
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5127 - loss: 0.6759 - val_accuracy: 0.6889 - val_loss: 0.6635
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5377 - loss: 0.6660 - val_accuracy: 0.7111 - val_loss: 0.6568
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5305 - loss: 0.6618 - val_accuracy: 0.7111 - val_loss: 0.6538
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5912 - loss: 0.6432 - val_accuracy: 0.6667 - val_loss: 0.6524
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4184 - loss: 0.6473 - val_accuracy: 0.5778 - val_loss: 0.6519
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4572 - loss: 0.6399 - val_accuracy: 0.6000 - val_loss: 0.6469
Epoch 9/10


In [194]:
from sklearn.metrics import classification_report

# Predict probabilities for the test set
y_pred_prob = model.predict(X_test)

# Convert probabilities to binary predictions using a threshold (e.g., 0.5)
y_pred = (y_pred_prob > 0.4).astype(int)

# Generate a classification report
report = classification_report(y_test, y_pred, target_names=label_cols)
print(report)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
                precision    recall  f1-score   support

     ABDOMINAL       0.60      1.00      0.75        34
    CREATININE       0.67      0.59      0.63        27
MAJOR-DIABETES       0.61      1.00      0.76        35
  ADVANCED-CAD       0.62      0.94      0.74        31

     micro avg       0.62      0.90      0.73       127
     macro avg       0.62      0.88      0.72       127
  weighted avg       0.62      0.90      0.72       127
   samples avg       0.61      0.90      0.69       127



C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## MultiLabelZeroShotGPTClassifier

In [195]:
# importing SKLLMConfig to configure OpenAI API (key and Name)
from skllm.config import SKLLMConfig

# Set your OpenAI API key
SKLLMConfig.set_openai_key("")
SKLLMConfig.set_openai_org("")

In [569]:
data_df1.head()

,text,ABDOMINAL,ADVANCED-CAD,ALCOHOL-ABUSE,ASP-FOR-MI,CREATININE,DIETSUPP-2MOS,DRUG-ABUSE,ENGLISH,HBA1C,KETO-1YR,MAJOR-DIABETES,MAKES-DECISIONS,MI-6MOS,preprocessed_text
0,Record date: 2106-02-12\n\nCampbell Orthopedic...,not met,met,not met,met,not met,met,not met,met,not met,not met,met,met,met,record date campbell orthopedic associate mad...
1,Record date: 2079-05-12\n\n\n\n\n\nMERCY CARE ...,not met,met,not met,met,not met,not met,not met,met,not met,not met,not met,met,not met,record date mercy care center mercy internal ...
2,Record date: 2120-09-19\n\nPersonal Data and O...,met,met,not met,met,not met,not met,not met,met,not met,not met,met,met,not met,record date personal data overall health pati...
3,Record date: 2067-11-24\n\n ...,not met,met,not met,met,not met,not met,not met,met,not met,not met,not met,met,not met,record date huntington emergency dept visit t...
4,Record date: 2094-02-16\n\nJENNIFER BOOKER\n\n...,not met,met,not met,not met,not met,not met,not met,met,met,not met,met,met,met,record date jennifer booker lc unit nashua m...


In [570]:
# Split data into training and testing sets
train, test = train_test_split(data_df1, random_state=42, test_size=0.20, shuffle=True)

# Chunked data containers (initialize empty lists)
X_train_chunks = train['preprocessed_text'].tolist()  # Use entire preprocessed_text for training
X_test_chunks = test['preprocessed_text'].tolist()  # Use entire preprocessed_text for testing

# Initialize Y values lists
y_train = []
y_test = []

# Extract Y values for training data
for _, row in train.iterrows():
    y_train_row = []
    for label in label_cols:
        if row[label] == 'met':
            y_train_row.append(label)
    y_train.append(y_train_row)

# Extract Y values for testing data
for _, row in test.iterrows():
    y_test_row = []
    for label in label_cols:
        if row[label] == 'met':
            y_test_row.append(label)
    y_test.append(y_test_row)


In [543]:
import pandas as pd
from skllm.models.gpt.vectorization import GPTVectorizer
from skllm.models.gpt.classification.zero_shot import MultiLabelZeroShotGPTClassifier
from sklearn.model_selection import train_test_split

# Feature extraction with GPT vectorizer
vectorizer = GPTVectorizer()
X_train_vec = vectorizer.fit_transform(X_train_chunks)
X_test_vec = vectorizer.transform(X_test_chunks)  # Use cleaned test data

Batch size: 1


100%|████████████████████████████████████████████████████████████████████████████████| 225/225 [04:45<00:00,  1.27s/it]


Batch size: 1


100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [01:19<00:00,  1.39s/it]


In [544]:
# Multi-label zero-shot classification
multi_label_zero_shot_clf = MultiLabelZeroShotGPTClassifier(max_labels=4)
multi_label_zero_shot_clf.fit(X_train_vec, label_cols)

# Predict labels for test data
multi_label_zero_shot_predictions = multi_label_zero_shot_clf.predict(X_test_vec)


100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [01:36<00:00,  1.69s/it]


#### Decoding output

In [565]:
def decode_predictions(predictions, label_cols):
    decoded_predictions = []
    for instance_pred in predictions:
        decoded_instance_pred = []
        for label, pred in zip(label_cols, instance_pred):
            if pred:
                decoded_instance_pred.append(label)
        decoded_predictions.append(decoded_instance_pred)
        #print("\n\n Decoded predictions:", decoded_instance_pred)
    return decoded_predictions

decoded_predictions = decode_predictions(multi_label_zero_shot_predictions, label_cols)


In [560]:
len(decoded_predictions)

57

In [571]:
len(y_test)

57

In [564]:
from sklearn.metrics import classification_report

# Convert decoded_predictions into the same format as the ground truth labels
decoded_predictions_flat = []
for predictions in decoded_predictions:
    decoded_predictions_flat.append([1 if label in predictions else 0 for label in label_cols])

# Convert test data labels into the same format
test_labels_flat = []
# Iterate over the rows of the DataFrame `test`
for _, row in test.iterrows():
    # Extract the labels from the row corresponding to `label_cols`
    labels = [1 if row[label] == 'met' else 0 for label in label_cols]
    # Append the labels for the current instance to the flattened list
    test_labels_flat.append(labels)

# Generate the classification report
report = classification_report(test_labels_flat, decoded_predictions_flat, target_names=label_cols)
print(report)


                precision    recall  f1-score   support

     ABDOMINAL       0.40      1.00      0.57        23
    CREATININE       0.43      0.91      0.58        22
MAJOR-DIABETES       0.51      0.92      0.66        26
  ADVANCED-CAD       0.67      0.67      0.67        30

     micro avg       0.48      0.86      0.62       101
     macro avg       0.50      0.87      0.62       101
  weighted avg       0.51      0.86      0.62       101
   samples avg       0.50      0.74      0.55       101



C:\Users\aslam\Anaconda3\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
